In [2]:
from SAC.Normal_Policy import Normal_Policy
from SAC.Value_function import Value_function
from SAC.SoftActorCritc import SoftActorCritic, plot

import time
import gym
import numpy as np

import sys
sys.path.insert(0, './../../laser-hockey-env')
import laser_hockey_env as lh
from importlib import reload

import progressbar
from tensorboardX import SummaryWriter


In [3]:
env_name = "Laser_hockey"
env = lh.LaserHockeyEnv()

In [4]:
ac_space = env.action_space
o_space = env.observation_space
print(ac_space.low)
print(o_space)

[-1. -1. -1. -1. -1. -1.]
Box(16,)


In [5]:
ac_space.shape

(6,)

In [6]:
direct = "C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/"
version = "v_full_10"
save_path_new = direct+version

In [7]:
q_fct_config = {"hidden_layers": [128, 128],"weights_init":None,"bias_init" :None}
v_fct_config = {"hidden_layers": [128, 128],"weights_init":None,"bias_init" :None}
pi_fct_config = {"hidden_layers": [128, 128], "dim":3,"weights_init":None,"bias_init" :None}

SAC_agent = SoftActorCritic(o_space=o_space, 
                            a_space=ac_space, 
                            value_fct=Value_function, 
                            policy_fct=Normal_Policy, 
                            env= env,
                            discount=  0.92, 
                            tau = 0.01, 
                            batch_size=128, 
                            alpha = 'auto',
                            lambda_V = 3e-4, 
                            lambda_Pi = 3e-4, 
                            lambda_Q = 3e-4,
                            dim_act = 3,
                            dim_obs = 16,
                            buffer_size = int(1e5),
                            q_fct_config=q_fct_config,
                            v_fct_config=v_fct_config,
                            pi_fct_config=pi_fct_config, 
                            save_path = save_path_new) 

In [8]:
player2 = lh.BasicOpponent()

In [9]:
writer = SummaryWriter('visualization/{}-{}'.format(env_name, version))

In [10]:

def train(iter_fit=1000, max_steps= 500,  env_steps = 1, grad_steps = 1, burn_in =5, writer = None):
        start = SAC_agent._sess.run(SAC_agent.global_step)
        saver = SAC_agent._saver
        bar = progressbar.ProgressBar(max_value=iter_fit)
        # Initilalize target V network
        SAC_agent._sess.run(SAC_agent._update_target_V_ops_hard)
        # Init Statistics      
          #moving_avg_winner =np.zeros(20)
        winner_count = 0
        draw_count = 0
        total = 0
        
        j = start
        for i in range(iter_fit):
            
            ob = env.reset()
            
            obs_agent2 = env.obs_agent_two()
            
           
            
            
            total_reward = 0
            
            #puck_hit = 0.0
            #puck_hit_once = False
            for _ in range(max_steps):
                for e_i in range(env_steps):#
                    
                    #env.render()
                    if j < burn_in:
                        a = env.action_space.sample()
                        a = a[:3]
                        #a = player2.act(ob) 
                           
                                        
                    else:
                        #print(SAC_agent.act_greedy(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"])))
                        a = SAC_agent.action(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))
                        a = a[0]
                    a2 = player2.act(obs_agent2)                    
                    comb_a = np.hstack([a,a2])
                    
                    
                    
                    (ob_new, reward, done, _info) = env.step(comb_a)
                     
                    #reward =  reward  + _info['reward_closeness_to_puck']  +  _info['reward_touch_puck']
                    reward = reward - (_info['reward_puck_direction'] == 0.0) * 0.01 #+ _info['reward_puck_direction']
                    total_reward += reward
                    
                    #SAC_agent.store_transition(ob, a, reward, ob_new, done)
                    SAC_agent.buffer.store(ob, a, reward, ob_new, done)
                    ob=ob_new
                    obs_agent2 = env.obs_agent_two()
  
                if j  >= 2:
                    for g_i in range(grad_steps):
                        update_value_target = False
                        if i % SAC_agent._config["target_update"] == 0:
                            update_value_target = True
                        loss_V_fct, loss_Q1_fct,loss_Q2_fct,loss_PI_fct= SAC_agent._train(update_value_target)

                        
                
                if done:
                    break 
            #moving_avg_winner[i%20]=((_info['winner']+1)/2.0)/20
            j += 1
            winner_count +=  _info['winner'] == 1
            draw_count += _info['winner'] == 0
            total += 1              
            SAC_agent._sess.run(SAC_agent.increment_global_step_op)
            #total_rewards_per_episode.append(total_reward)
            #if i % 1 == 0:
            if i % 250 == 0:
                saver.save(SAC_agent._sess, save_path_new)

                #plot(total_rewards_per_episode,winning=winner, plot_type = 3) 
            if writer: 
                writer.add_scalar('win_loss_ratio/wins',winner_count/total , start+i)
                writer.add_scalar('win_loss_ratio/draws',draw_count/total , start+i)
                writer.add_scalar('reward',total_reward , start+i)
            bar.update(i)   
        return winner_count

In [ ]:
train(10000, burn_in = 0, writer = writer)

  0% (4 of 10000) |                      | Elapsed Time: 0:00:20 ETA:  19:56:11

In [10]:
SAC_agent._saver.save(SAC_agent._sess, save_path_new)

'C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/v_full_9'

In [15]:

for i in range(10):
    ob = env.reset()
    for _ in range(1000):
        env.render()
#        distance_to_opponent = np.sqrt((ob[6] -ob[0])**2 + (ob[7] -ob[1])**2)
 #       distance_to_puck = np.sqrt((ob[12] -ob[0])**2 + (ob[13] -ob[1])**2)
  #      angle_to_opponent = np.arctan2((ob[7] -ob[1]),(ob[6] -ob[0])) - ob[2]
   #     angle_to_puck = np.arctan2((ob[13] -ob[1]),(ob[12] -ob[0])) - ob[2]
    #    ob = np.append(ob, [distance_to_opponent, distance_to_puck,angle_to_opponent, angle_to_puck] )
        obs_agent2 = env.obs_agent_two()    
        a = SAC_agent.action(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))
        a1 = a[0]
        #print(a1, SAC_agent.act_greedy(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))[0])
        #a1 = player2.act(ob)
        
        a2 = player2.act(obs_agent2)
        
        ob, r, d, info = env.step(np.hstack([a1,a2]))    
        #print(info)
        obs_agent2 = env.obs_agent_two()
        if d: break
env.close()

In [8]:
env.close()